# Problem 1
Let $w_1, \dots, w_n$ such that $\Pi_{i \in \emptyset} w_i = 1$

$$
S \;=\;\sum_{A,B \subseteq \{1,\dots,n\}}
  \frac{\displaystyle \prod_{i\in A} w_i}
       {\displaystyle \prod_{i\in B} w_i}\,.
$$


In [23]:
using BenchmarkTools
v = rand(15)

15-element Vector{Float64}:
 0.3064522625937147
 0.8483778114957404
 0.959154820374537
 0.30040458432953254
 0.7418652036519456
 0.06989906575852844
 0.9959780881042372
 0.48376983796669715
 0.9853287987763725
 0.11772253244120945
 0.6446827364432318
 0.3768793576297751
 0.22307602454774544
 0.20566715542615088
 0.01238254513311321

In [24]:
# Chatgpt gen
function brute_force_pairsum(w)
    """
    Brute-force summation over all pairs of subsets A,B ⊆ {1,...,n},
    returning sum(prod(w[i] for i in A) / prod(w[i] for i in B)).
    This has time complexity O(4^n).
    """
    n = length(w)
    total_sum = 0.0

    # For each subset A, represented by an integer bitmask a:
    for a in 0:(2^n - 1)
        # Compute product of w[i] for i in A
        prodA = 1.0
        # We interpret the bits of 'a' to decide membership in A
        # (i.e. if the i-th bit is set, i is in A).
        for i in 0:(n-1)
            if ((a >> i) & 1) == 1
                prodA *= w[i+1]  # +1 since i is 0-based
            end
        end

        # For each subset B, represented by an integer bitmask b:
        for b in 0:(2^n - 1)
            # Compute product of w[i] for i in B
            prodB = 1.0
            for j in 0:(n-1)
                if ((b >> j) & 1) == 1
                    prodB *= w[j+1]
                end
            end

            # Add the ratio to the grand total
            total_sum += prodA / prodB
        end
    end

    return total_sum
end

@btime brute_force_pairsum(v)

  53.758 s (1 allocation: 16 bytes)


2.1652753812260576e12

In [25]:
function sum_prod(w)
    if length(w) == 1
        w = w[1]
        return 2 + w + 1/w
    end

    return sum_prod(w[end]) * sum_prod(w[1:end-1])
end

@btime sum_prod(v)


  238.133 ns (29 allocations: 1.66 KiB)


2.1652753817575032e12

# Testing

In [26]:
for i in 1:200
    v = rand(10)

    if brute_force_pairsum(v) ≉ sum_prod(v)
        println("$(brute_force_pairsum(v)), $(sum_prod(v)), $v")
    end
end